In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline

In [2]:
# veri kaynağı: http://veribilimi.co/data/iris.csv
df = pd.read_csv("D:/Datasets/SosyalMedyaReklamKampanyasi.csv")

# 1. Veri Keşfi

In [3]:
df.head()

KullaniciID Cinsiyet  Yas  TahminiMaas  SatinAldiMi
0     15624510    Erkek   19        19000            0
1     15810944    Erkek   35        20000            0
2     15668575    Kadın   26        43000            0
3     15603246    Kadın   27        57000            0
4     15804002    Erkek   19        76000            0

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
KullaniciID    400 non-null int64
Cinsiyet       400 non-null object
Yas            400 non-null int64
TahminiMaas    400 non-null int64
SatinAldiMi    400 non-null int64
dtypes: int64(4), object(1)
memory usage: 15.7+ KB


In [5]:
df.describe()

KullaniciID         Yas    TahminiMaas  SatinAldiMi
count  4.000000e+02  400.000000     400.000000   400.000000
mean   1.569154e+07   37.655000   69742.500000     0.357500
std    7.165832e+04   10.482877   34096.960282     0.479864
min    1.556669e+07   18.000000   15000.000000     0.000000
25%    1.562676e+07   29.750000   43000.000000     0.000000
50%    1.569434e+07   37.000000   70000.000000     0.000000
75%    1.575036e+07   46.000000   88000.000000     1.000000
max    1.581524e+07   60.000000  150000.000000     1.000000

In [6]:
# Veri keşfi ile başka birçok groupby, count_values vb. sorgular kullanılabilir. Grafikler çizilip incelenebilr.

# 2. Veri Temizliği ve Ön Hazırlığı

In [7]:
# Veri önceden temizlenmiş olduğundan bu aşamada birşey yapmıyoruz.Boşluk yok aykırı değer yok.

## 2.1. Veriyi X,y olarak bölme (nitelikler matrisi ve hedef değişken - dikey bölme)

In [8]:
# Hedef değişken en sonda olduğu için ve sadece nümerik nitelikleri kullanacağımız için
# baştan (0) son indekse kadar tüm nitelikleri alıyoruz
X = df.iloc[:,1:-1].values
X.shape

(400, 3)

In [9]:
# Kontrol
X[:3]

array([['Erkek', 19, 19000],
       ['Erkek', 35, 20000],
       ['Kadın', 26, 43000]], dtype=object)

In [10]:
# En son indeks hedef değiken (output)
y = df.iloc[:,-1].values
y.shape

(400,)

In [11]:
# Kontrol
y[:10]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int64)

## 2.2. Kategorik Nitelik Dönüştürme

### 2.2.1. Girdi Nitelikler için LabelEncoder

In [12]:
# Her bir kategorik nitelik için ayrı bir LabelEncoder nesnesi yaratalım. 
# Tek bir nesne ile de yapabiliriz ancak bu durumda orjinal değerlere dönmek mümkün olmaz.
from sklearn.preprocessing import LabelEncoder
labelencoder_cinsiyet = LabelEncoder()

In [13]:
# Her bir kategorik nitelik için oluşturduğumuz LabelEncoder nesnelerini (makine) eğitip sonucu dönüştürüp string değerler
# ile dğiştiriyoruz. Artık X içinde string değer kalmıyor. Onların yerine nümerik karşılıkları var. Bu nümerik değerlerin
# orjinal veride hangi string değere karşılık geldiğini ilgili LabelEncoder nesnesi biliyor.
X[:,0] = labelencoder_cinsiyet.fit_transform(X[:,0])

### 2.2.2. Hedef değişken için LabelEncoder

In [14]:
# Hedef değişken 0 ve 1 olarak kodlanmış

### 2.2.3. One Hot Encoder - ColumnTransformer

In [15]:
from sklearn.preprocessing import OneHotEncoder

In [16]:
onehotencoder = OneHotEncoder(handle_unknown='ignore', categories='auto')

In [17]:
# Birden sütunu ilgilendiren dönüşümleri kolay yapabilmek için kullanışlı bir sınıf (Estimator kökenli)
# Biz burada sadece OneHotEncoder için kullanacağız.
from sklearn.compose import ColumnTransformer

In [18]:
# one_hot_encode_with_ct bizim kendi verdiğiiz isimdir. onehotencoder'ı yukarıda yarattık. [0,1,2,3,4,5,6,7,8] ise dönüşecek
# niteliklerin indeks değerleridir.
ct = ColumnTransformer(
    [('one_hot_encode_with_ct', onehotencoder, [0]),], 
    remainder='passthrough'
)

In [19]:
# ColumnTransformer nesnesi ile onehotencoder nesnesini eğitip gerekli dönüşümleri gerçekleştirip X'e tekrar atayalım.
# Burada X'in boyutu(shape) değişecektir.
X = ct.fit_transform(X)

In [20]:
# Boyut değişmiş olmalı. Çnkü gölge değişkenler eklendi
X.shape

(400, 4)

In [21]:
X[:3,:]

array([[1.0, 0.0, 19, 19000],
       [1.0, 0.0, 35, 20000],
       [0.0, 1.0, 26, 43000]], dtype=object)

## 2.3. Feature Scaling (Standardizasyon)

In [22]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(X)

## 2.4. Veriyi eğitim ve test setlerine bölme (yatay bölme)

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=142)

In [25]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(300, 4)
(300,)
(100, 4)
(100,)


# 3. Model Oluşturma ve Eğitim

In [28]:
from sklearn.svm import SVC

In [29]:
classifier = SVC(random_state=142, gamma='auto')

In [30]:
classifier.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=142, shrinking=True, tol=0.001,
    verbose=False)

# 4. Predictions

In [31]:
y_pred = classifier.predict(X_test)

## 4.1. Tahmin sonuçlarını numpy.ndarray'den dataframe yapma (İncelemek daha Kolay)

In [32]:
predictions = pd.DataFrame(data=y_pred,    # values
                index=range(len(y_pred)),    # 1st column as index
                   columns=['y_pred'])  # 1st row as the column names

# Sadece y_pred'den oluşan df'e test(gerçek) y_test'i sütun olarak ekleme
predictions['y_test'] = y_test
predictions.head()

y_pred  y_test
0       1       1
1       1       1
2       0       0
3       1       0
4       1       0

# 5. Model Değerlendirme

## 5.1. Hata Matrisi (Confusion Matrix)

In [33]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(cm)

[[58  9]
 [ 6 27]]


## 5.2. Sınıflandırma Performansı Değerlendirme (Accuracy)

In [34]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: % {:10.2f}".format(accuracy*100)) 
# :10.2f ifadesi ondalık kısımda iki basamak göstermek için

Accuracy: %      85.00


# 6. Hyperparameter Tuning - Model Seçme - En İyi Hiperparametreleri Bulma

In [35]:
# mevcut parametreler
classifier.get_params

<bound method BaseEstimator.get_params of SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=142, shrinking=True, tol=0.001,
    verbose=False)>

## 6.1. Hiperparametre kombinasyonlarını belirleme

In [36]:
param_grid = [
  {
    'C': [1.0, 10.0, 100.0,1000.0], 
   'kernel': ['linear']
    },
    {
     'C': [1.0, 10.0, 100.0,1000.0], 
   'kernel': ['rbf'],
   'gamma':[0.5, 0.1, 0.01, 0.001, 0.0001]
    }
 ]

## 6.2. GridSearchCV ile en iyi hiperparametre setini/modeli bulma

In [38]:
from sklearn.model_selection import GridSearchCV

In [39]:
grid_search = GridSearchCV(estimator=classifier, 
                           param_grid=param_grid,
                          scoring='accuracy',
                          cv=5,
                          n_jobs=4,
                          iid=False)
# maksimum için n_jobs=-1 kullanılır.
# iid=False deprecation ikazını kapatmak için eklendi

In [40]:
grid_search = grid_search.fit(X_train, y_train)

## 6.3. En iyi skor, hiperparametre ve model

In [41]:
best_score = grid_search.best_score_
best_score

0.9233333333333335

In [42]:
best_params = grid_search.best_params_
best_params

{'C': 100.0, 'gamma': 0.1, 'kernel': 'rbf'}

In [43]:
best_estimator = grid_search.best_estimator_
best_estimator

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=142, shrinking=True, tol=0.001,
    verbose=False)

# 7. En iyi Model Performansı

In [44]:
y_pred = best_estimator.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: % {:10.2f}".format(accuracy*100)) 
# :10.2f ifadesi ondalık kısımda iki basamak göstermek için

Accuracy: %      86.00


# 8. Modeli Saklama ve Tekrar Kullanma

In [73]:
# Saklamadan önce tüm veri seti ile modeli en iyi hiperparametrelerle tekrar eğitelim

## 8.1. Tüm veri ile modeli son olarak eğitme

In [45]:
# Boş bir sınıflandırıcı (SVC) nesnesi oluşturalım
classifier_last = SVC(random_state=142, gamma='auto')

In [46]:
# Boş nesneye en iyi modelin hiperparametrelerini aktaralım
classifier_last.set_params = best_estimator.get_params

In [47]:
# En iyi hiperparametrelerle sınıflandırıcıyı/estimator/modeli eğitelim
classifier_last.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=142, shrinking=True, tol=0.001,
    verbose=False)

## 8.2. Modeli kaydetme

In [48]:
import joblib

In [49]:
joblib.dump(classifier_last, 
"D:/OneDrive/egitim/verilen/machine-learning-with-python/10_classification/saved_models/06.model_selection_tuning_with_SosyalMedyaReklamKampanyası_data.pkl")

['D:/OneDrive/egitim/verilen/machine-learning-with-python/10_classification/saved_models/06.model_selection_tuning_with_SosyalMedyaReklamKampanyası_data.pkl']

## 8.3. Model Okuma

In [50]:
classifier_last_loaded = joblib.load(
"D:/OneDrive/egitim/verilen/machine-learning-with-python/10_classification/saved_models/06.model_selection_tuning_with_SosyalMedyaReklamKampanyası_data.pkl")

In [51]:
classifier_last_loaded.get_params

<bound method BaseEstimator.get_params of SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=142, shrinking=True, tol=0.001,
    verbose=False)>